<a href="https://colab.research.google.com/github/lsgotti/TCC_Python/blob/main/Simplificado_Undersampling_online_intervalo_de_500.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#carrega imports e instala o scikit-multiflow
!pip install -U scikit-multiflow
# Imports
from skmultiflow.data import FileStream
from skmultiflow.data import SEAGenerator
from skmultiflow.evaluation import EvaluatePrequential
from skmultiflow.bayes import NaiveBayes
from skmultiflow.trees import HoeffdingTreeClassifier
from skmultiflow.trees import HoeffdingAdaptiveTreeClassifier
from skmultiflow.trees import ExtremelyFastDecisionTreeClassifier
from skmultiflow.meta import LearnPPNSEClassifier
from skmultiflow.drift_detection import ADWIN
from skmultiflow.meta import OzaBaggingClassifier

from sklearn.linear_model import SGDClassifier

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
#carrega imports
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import svm
import pandas as pd
from sklearn import datasets, linear_model
from matplotlib import pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from skmultiflow.lazy import KNNClassifier

from google.colab import files

import numpy as np
from skmultiflow.data.file_stream import FileStream
from skmultiflow.data.data_stream import DataStream
from skmultiflow.trees import HoeffdingTreeClassifier
from skmultiflow.evaluation import EvaluatePrequential
import pandas as pd
import io
from google.colab import files
from skmultiflow.trees import HoeffdingTree
from skmultiflow.meta import AdaptiveRandomForestClassifier
from datetime import datetime, date

In [ ]:
#segunda parte
#importa arquivo para teste
uploaded = files.upload()
#Importa o arquivo para o colab
import io

#abre o arquivo no dataframe
df = pd.read_csv(io.BytesIO(uploaded['tomcat2.csv']))

Saving fabric82.csv to fabric82.csv


In [ ]:

np.set_printoptions(formatter={'float_kind':'{:25f}'.format})

#Etapa de downsampling

from sklearn.utils import resample


#Divide o dataframe em codigos 0 e 1, apenas o 1 sera usado

m=df.code==0
df_code0=df[m]
df_code1=df[~m]

print(df_code1.contains_bug.value_counts())

# Separa classes minoritarias e majoritarias
df_majority = df_code1[df_code1.contains_bug==0]
df_minority = df_code1[df_code1.contains_bug==1]


# Downsample de classe majoritaria
df_majority_downsampled = resample(df_majority, replace=False, n_samples=len(df_minority), random_state=123)

# Combina classe minoritaria com downsample da majoritaria
df_downsampled_partial = pd.concat([df_majority_downsampled, df_minority])

print(df_downsampled_partial.contains_bug.value_counts())
df_downsampled = pd.concat([df_downsampled_partial, df_code0])

# novo count de classes
print(df_downsampled.contains_bug.value_counts())


0    12203
1     5180
Name: contains_bug, dtype: int64
0    5180
1    5180
Name: contains_bug, dtype: int64
0    15697
1    10360
Name: contains_bug, dtype: int64


,author_date_unix_timestamp,fix,ns,nd,nf,entrophy,la,ld,lt,ndev,age,nuc,exp,rexp,sexp,contains_bug,original_timestamp,code
29654,1448288139,False,10,52,52,5.690185,53,53,155.884615,54,0.011111,12242,6898.5,8.854307,110,0,1.456928e+09,1
15245,1389179379,False,1,1,1,0.000000,2,1,402.000000,24,9.205590,14,5910.0,1.108630,3886,0,1.397819e+09,1
28037,1442403718,False,1,1,1,0.000000,3,5,38.000000,53,62.952685,2,14136.0,1.015885,104,0,1.451044e+09,1
31901,1468825139,False,1,1,1,0.000000,2,2,1848.000000,55,3.022431,2139,1024.0,1.330860,220,0,1.477465e+09,1
6803,1342082715,False,1,1,1,0.000000,4,0,78.000000,21,16.745972,26,5648.0,1.059716,1085,0,1.350723e+09,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33070,1510904247,False,1,1,1,0.000000,1,1,0.000000,1,0.003576,1,1.0,280.611650,1,0,1.510904e+09,0
33072,1512379324,False,0,0,0,0.000000,0,0,0.000000,0,0.000000,0,0.0,0.000000,0,0,1.512379e+09,0
33074,1512468949,False,1,1,1,0.000000,2,2,0.000000,1,0.000000,0,0.0,0.000000,0,0,1.512469e+09,0
33076,1512468986,False,1,1,1,0.000000,1,1,0.000000,1,0.000000,0,1.0,0.000000,0,0,1.512469e+09,0


In [ ]:
#embaralha o dataframe
df_downsampled = df_downsampled.sample(frac=1)

#Abre stream com dataframe downsampled
stream = DataStream(df_downsampled)

#embaralha o dataframe
#df_downsampled = df_downsampled.sample(frac=1)

#Abre stream com dataframe downsampled
stream = DataStream(df_downsampled)

#Inicializa variaveis e lista que serão usadas, deve ser rodado a cada teste

indexStream=1

periodicidadeTreinamento = 500

w = 100

PREDICT = 0

TRAIN = 1

#Escolher o classificador

#classifier = HoeffdingTree()
classifier = OzaBaggingClassifier(base_estimator=HoeffdingTree(), n_estimators=20)
#classifier = LearnPPNSEClassifier()
#classifier = HoeffdingAdaptiveTreeClassifier()
#classifier = ExtremelyFastDecisionTreeClassifier()
#classifier = AdaptiveRandomForestClassifier(n_estimators=100)
#classifier = KNNClassifier()
correctness_dist = []
i = 0

poolCleanInstances = []
poolDefectInducingInstances = []
poolTrainInstances = []
poolTrainClasses = []
poolTrainInstances2= []

#Cria a classe de recall
class Recall:
  recall0 = 0
  recall1 = 0

  b0 = 0
  b1 = 0

  alpha = 0.999

  def updateRec0(self, value):
    self.recall0 = value + self.alpha * self.recall0
    self.b0 =  self.alpha * self.b0 + 1.0;

  def getRecall0(self):
    if self.b0 > 0:
      #print("b0>0")
      #print(self.b0)
      return self.recall0/self.b0
    else:
      #print("b0 not 0")
      #print(self.b0)
      return 0

  def updateRec1(self, value):
    self.recall1 = value +  self.alpha * self.recall1
    self.b1 =  self.alpha * self.b1 + 1.0;

  def getRecall1(self):
    if self.b1 > 0:
      #print("b1>0")
      #print(self.b1)
      return self.recall1/self.b1
    else:
      #print("b1 not 0")
      #print(self.b1)
      return 0

#Cria as listas de recall
import math

recall = Recall()

recall0list = [0]
recall1list = [0]

TP = 0
TN = 0
FP = 0
FN = 0

isFirstInstance = True
#Roda enquanto ainda tem instancias na stream
while (stream.has_more_samples()):
  #Inicializa o classificador caso seja a primeira instancia
  if(isFirstInstance and len(poolTrainInstances) > 0):
    inst = poolTrainInstances[0]

    inst2 = np.array([inst[0,1:14],inst[0,1:14]])
    classe2 = np.array([0,1])

    classifier.partial_fit(inst2, classe2,classes=stream.target_values)
    isFirstInstance = False
  #Incrementa o indexStream para comparar com o w e treinar classificador
  indexStream = indexStream +1
  #X são as caracteristicas da instancia e control é o code (0 ou 1)
  X, control = stream.next_sample()
  current_timestamp = X[0,0]
  original_timestamp = X[0,16]
  current_date = datetime.fromtimestamp(current_timestamp)
  original_date = datetime.fromtimestamp(original_timestamp)
  classe = int(X[0,15])
  X2 = [X[0 ,1:14]]

  #Se control for predict
  if(control == PREDICT):
    #usa o metodo predict e guarda em prediction
    prediction = classifier.predict(X2)
    #caso prediction acerte, faz update nas listas e no recall
    if classe == prediction[0]:
      correctness_dist.append(1)
      #Entra se a classe eh 0 e da update no recall 0
      if (classe == 0):
        if not recall1list:
          recall1list.append(0)
        if recall1list:
          recall1list.append(recall1list[-1])
        TN = TN + 1
        #print("TN ACERTOU")
        recall.updateRec0(1)
        recall0list.append(recall.getRecall0())
        #caso prediction erre, faz update nas listas e no recall
      #Entra se a classe eh 1 e da update no recall 1
      else:
        if not recall1list:
          recall1list.append(0)
        if recall1list:
          recall1list.append(recall1list[-1])
        TP = TP + 1
        recall.updateRec1(1)
        recall1list.append(recall.getRecall1())

        #print("TP ACERTOU")
    #caso prediction acerte, faz update nas listas e no recall
    else:
      correctness_dist.append(0)
      #Entra se a classe eh 0 e da update no recall 0
      if (classe == 0):
        FN = FN + 1
        recall.updateRec0(0)
        recall0list.append(recall.getRecall0())
        #print("FN ERROU")
      #Entra se a classe eh 1 e da update no recall 1
      else:
        FP = FP + 1
        recall.updateRec1(0)
      recall1list.append(recall.getRecall1())
        #print("FN ERROU")

    #poolCleanInstances.append(X)
  #se control for train
  if(control == TRAIN):
    #adiciona a instancia na lista de instancias a serem treinadas, essas instancias so serao treinadas quando indexStream chegar no valor de periodicidadeTreinamento
    poolTrainInstances.append(X)
    if(X[0,2] == 0):
           # update the tree
      poolCleanInstances.append(X)
    if(X[0,2] == 1):
      poolDefectInducingInstances.append(X)

  #Caso indexStream chegue no valor de periodicidadeTreinamento, realizamos o treinamento percorrendo todas as instancias na lista poolTrainInstances e treinando o classificador com partial_fit
  if(indexStream % periodicidadeTreinamento == 0):
    #resetar  o classificador
    #nao resetar o pool de instancias
    #treinar o classificador com ten fold cross validation
    #fazer isso pra 100, 500 e 1000 periodicidade
    #
    for inst in poolTrainInstances:

      inst2 = np.array([inst[0,1:14]])
      classe2 = np.array([int(inst[0,15])])

      classifier.partial_fit(inst2, classe2)
    indexStream=1
    poolTrainInstances.clear()

print("TP: "+str(TP))
print("FP: "+str(FP))
print("TN: "+str(TN))
print("FN: "+str(FN))
print(len(correctness_dist))
print(sum(correctness_dist))
print(classifier)

#Organiza os recalls em uma lista e calcula gmean e por ultimo baixa como csv
for i in range(abs(len(recall0list) - len(recall1list))):
  if len(recall0list) > len(recall1list):
    recall1list.append(recall1list[-1])
  else:
    recall0list.append(recall0list[-1])

print(len(recall1list))
print(len(recall0list))
print(len(recall1list)+len(recall0list))

gmeanList = []
for i in range(len(recall0list)):
  gmeanList.append(math.sqrt(recall0list[i] * recall1list[i]))

print(len(gmeanList))

from google.colab import files

results = pd.DataFrame(
    {'recall0': recall0list,
     'recall1': recall1list,
     'gmean': gmeanList
    })

results.to_csv('results.csv')
files.download('results.csv')